# Esplorazione del Sentiment con 7 Classi di Emozioni

Questo notebook estende l'analisi del sentiment originale implementando un sistema di classificazione a 7 classi invece di 3. L'obiettivo è ottenere una granularità maggiore nell'analisi delle emozioni, distinguendo tra livelli di intensità diversi del sentiment.

### Le 7 Classi di Emozioni:

1. **Estremamente Negativo** (Strongly Negative / Extremely Negative)
2. **Negativo** (Negative)
3. **Un po' Negativo** (Somewhat Negative)
4. **Neutro** (Neutral)
5. **Un po' Positivo** (Somewhat Positive)
6. **Positivo** (Positive)
7. **Estremamente Positivo** (Strongly Positive / Extremely Positive)

### Approccio:

- **Fase 1:** Analisi della distribuzione dei punteggi compound nel dataset
- **Fase 2:** Test di diverse configurazioni di soglie
- **Fase 3:** Selezione della configurazione ottimale basata sulla distribuzione
- **Fase 4:** Applicazione della classificazione e salvataggio dei risultati

### Configurazioni di Soglie da Testare:

**Configurazione 1 - Divisione Uniforme:**

- Estremamente Negativo: compound ≤ -0.67
- Negativo: -0.67 < compound ≤ -0.33
- Un po' Negativo: -0.33 < compound ≤ -0.05
- Neutro: -0.05 < compound ≤ 0.05
- Un po' Positivo: 0.05 < compound ≤ 0.33
- Positivo: 0.33 < compound ≤ 0.67
- Estremamente Positivo: compound > 0.67

**Configurazione 2 - Concentrata sul Neutro:**

- Estremamente Negativo: compound ≤ -0.75
- Negativo: -0.75 < compound ≤ -0.4
- Un po' Negativo: -0.4 < compound ≤ -0.1
- Neutro: -0.1 < compound ≤ 0.1
- Un po' Positivo: 0.1 < compound ≤ 0.4
- Positivo: 0.4 < compound ≤ 0.75
- Estremamente Positivo: compound > 0.75

**Configurazione 3 - Basata su Percentili:**
Verrà calcolata automaticamente in base alla distribuzione effettiva dei dati.


In [37]:
# Importazione delle librerie necessarie
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib

matplotlib.use("Agg")
from collections import Counter
import csv

In [38]:
# Configurazione: Seleziona modalità (train, val, test)
mode = "val"  # Cambia in 'val' o 'test' secondo necessità

# Percorsi dei file
excel_path = f"../../data/raw/{mode}/how2sign_{mode}.xlsx"
video_folder = f"../../data/raw/{mode}/raw_videos_front_{mode}"

print(f"Modalità selezionata: {mode}")
print(f"Percorso Excel: {excel_path}")
print(f"Cartella video: {video_folder}")

Modalità selezionata: val
Percorso Excel: ../../data/raw/val/how2sign_val.xlsx
Cartella video: ../../data/raw/val/raw_videos_front_val


## Fase 1: Caricamento e Preprocessing dei Dati


In [39]:
# Caricamento del file Excel
print("Caricamento del file Excel...")
df = pd.read_excel(excel_path)

print(f"\nColonne disponibili: {list(df.columns)}")
print(f"Numero totale di righe: {len(df)}")

# Estrazione delle frasi (ultima colonna)
sentences = df.iloc[:, -1].dropna().tolist()

# Estrazione dei nomi dei video
if "SENTENCE_NAME" in df.columns:
    video_names = df["SENTENCE_NAME"].dropna().tolist()
else:
    raise ValueError("Colonna SENTENCE_NAME non trovata nel dataset")

# Filtraggio: solo video esistenti con captions valide
existing_videos = set(os.listdir(video_folder))
video_caption_mapping = {}

for video, caption in zip(video_names, sentences):
    video_file = f"{video}.mp4"
    if video_file in existing_videos and pd.notna(caption):
        video_caption_mapping[video] = caption

print(f"\nCreato mapping per {len(video_caption_mapping)} video-caption pairs")
print("\nEsempi di mapping:")
for i, (video, caption) in enumerate(list(video_caption_mapping.items())[:3]):
    print(f"{i+1}. Video: {video}")
    print(f"   Caption: {caption[:100]}...")

Caricamento del file Excel...

Colonne disponibili: ['VIDEO_ID', 'VIDEO_NAME', 'SENTENCE_ID', 'SENTENCE_NAME', 'START', 'END', 'SENTENCE']
Numero totale di righe: 1741

Creato mapping per 1739 video-caption pairs

Esempi di mapping:
1. Video: -d5dN54tH2E_0-1-rgb_front
   Caption: We're going to work on a arm drill that will help you have graceful hand movements in front of you....
2. Video: -d5dN54tH2E_1-1-rgb_front
   Caption: I call it painting the wall....
3. Video: -d5dN54tH2E_10-1-rgb_front
   Caption: So we're going to go up and down...

Colonne disponibili: ['VIDEO_ID', 'VIDEO_NAME', 'SENTENCE_ID', 'SENTENCE_NAME', 'START', 'END', 'SENTENCE']
Numero totale di righe: 1741

Creato mapping per 1739 video-caption pairs

Esempi di mapping:
1. Video: -d5dN54tH2E_0-1-rgb_front
   Caption: We're going to work on a arm drill that will help you have graceful hand movements in front of you....
2. Video: -d5dN54tH2E_1-1-rgb_front
   Caption: I call it painting the wall....
3. Video: -d5dN54

## Fase 2: Calcolo dei Punteggi di Sentiment con VADER


In [40]:
# Inizializzazione dell'analizzatore VADER
analyzer = SentimentIntensityAnalyzer()

# Calcolo del sentiment per ogni video-caption pair
video_sentiment_data = []

for video_name, caption in video_caption_mapping.items():
    scores = analyzer.polarity_scores(caption)
    compound_score = scores["compound"]

    video_sentiment_data.append(
        {
            "video_name": video_name,
            "caption": caption,
            "compound_score": compound_score,
            "scores": scores,
        }
    )

print(f"Calcolato sentiment per {len(video_sentiment_data)} video")

# Estrazione dei compound scores per analisi statistiche
compound_scores = [item["compound_score"] for item in video_sentiment_data]

print(f"\nStatistiche dei compound scores:")
print(f"Media: {np.mean(compound_scores):.4f}")
print(f"Mediana: {np.median(compound_scores):.4f}")
print(f"Deviazione standard: {np.std(compound_scores):.4f}")
print(f"Min: {np.min(compound_scores):.4f}")
print(f"Max: {np.max(compound_scores):.4f}")

Calcolato sentiment per 1739 video

Statistiche dei compound scores:
Media: 0.1615
Mediana: 0.0000
Deviazione standard: 0.3208
Min: -0.8555
Max: 0.9475


## Fase 3: Analisi della Distribuzione dei Compound Scores


In [41]:
# Visualizzazione della distribuzione con istogramma dettagliato
plt.figure(figsize=(14, 6))

# Subplot 1: Istogramma con molti bin
plt.subplot(1, 2, 1)
plt.hist(compound_scores, bins=50, color="skyblue", edgecolor="black", alpha=0.7)
plt.title(
    "Distribuzione dei Punteggi Compound (50 bins)", fontsize=12, fontweight="bold"
)
plt.xlabel("Valore Compound", fontsize=10)
plt.ylabel("Frequenza", fontsize=10)
plt.axvline(x=0, color="red", linestyle="--", linewidth=1, label="Zero")
plt.grid(axis="y", alpha=0.3)
plt.legend()

# Subplot 2: Box plot per vedere outliers e quartili
plt.subplot(1, 2, 2)
plt.boxplot(compound_scores, vert=True)
plt.title("Box Plot dei Punteggi Compound", fontsize=12, fontweight="bold")
plt.ylabel("Valore Compound", fontsize=10)
plt.axhline(y=0, color="red", linestyle="--", linewidth=1, label="Zero")
plt.grid(axis="y", alpha=0.3)

plt.tight_layout()

# Salvataggio del grafico
output_folder = "../../reports/figures"
os.makedirs(output_folder, exist_ok=True)
plt.savefig(
    os.path.join(output_folder, f"{mode}_compound_distribution_analysis.png"),
    dpi=300,
    bbox_inches="tight",
)
plt.close()

print("Grafico della distribuzione salvato!")

Grafico della distribuzione salvato!


In [42]:
# Calcolo dei percentili per la Configurazione 3
percentiles = [0, 14.28, 28.57, 42.86, 57.14, 71.43, 85.71, 100]
percentile_values = np.percentile(compound_scores, percentiles)

print("\nConfigurazione 3 - Soglie Basate sui Percentili:")
print("=" * 60)
for i, (p, val) in enumerate(zip(percentiles, percentile_values)):
    print(f"Percentile {p:.2f}%: {val:.4f}")

print("\nSoglie calcolate:")
print(f"Estremamente Negativo: compound ≤ {percentile_values[1]:.4f}")
print(f"Negativo: {percentile_values[1]:.4f} < compound ≤ {percentile_values[2]:.4f}")
print(
    f"Un po' Negativo: {percentile_values[2]:.4f} < compound ≤ {percentile_values[3]:.4f}"
)
print(f"Neutro: {percentile_values[3]:.4f} < compound ≤ {percentile_values[4]:.4f}")
print(
    f"Un po' Positivo: {percentile_values[4]:.4f} < compound ≤ {percentile_values[5]:.4f}"
)
print(f"Positivo: {percentile_values[5]:.4f} < compound ≤ {percentile_values[6]:.4f}")
print(f"Estremamente Positivo: compound > {percentile_values[6]:.4f}")


Configurazione 3 - Soglie Basate sui Percentili:
Percentile 0.00%: -0.8555
Percentile 14.28%: 0.0000
Percentile 28.57%: 0.0000
Percentile 42.86%: 0.0000
Percentile 57.14%: 0.0772
Percentile 71.43%: 0.3612
Percentile 85.71%: 0.5419
Percentile 100.00%: 0.9475

Soglie calcolate:
Estremamente Negativo: compound ≤ 0.0000
Negativo: 0.0000 < compound ≤ 0.0000
Un po' Negativo: 0.0000 < compound ≤ 0.0000
Neutro: 0.0000 < compound ≤ 0.0772
Un po' Positivo: 0.0772 < compound ≤ 0.3612
Positivo: 0.3612 < compound ≤ 0.5419
Estremamente Positivo: compound > 0.5419


## Fase 4: Test delle Tre Configurazioni di Soglie


In [43]:
# Definizione delle tre configurazioni di soglie
configurations = {
    "Config 1 - Uniforme": {
        "Extremely Negative": (-1.0, -0.67),
        "Negative": (-0.67, -0.33),
        "Somewhat Negative": (-0.33, -0.05),
        "Neutral": (-0.05, 0.05),
        "Somewhat Positive": (0.05, 0.33),
        "Positive": (0.33, 0.67),
        "Extremely Positive": (0.67, 1.0),
    },
    "Config 2 - Concentrata": {
        "Extremely Negative": (-1.0, -0.75),
        "Negative": (-0.75, -0.4),
        "Somewhat Negative": (-0.4, -0.1),
        "Neutral": (-0.1, 0.1),
        "Somewhat Positive": (0.1, 0.4),
        "Positive": (0.4, 0.75),
        "Extremely Positive": (0.75, 1.0),
    },
    "Config 3 - Percentili": {
        "Extremely Negative": (percentile_values[0], percentile_values[1]),
        "Negative": (percentile_values[1], percentile_values[2]),
        "Somewhat Negative": (percentile_values[2], percentile_values[3]),
        "Neutral": (percentile_values[3], percentile_values[4]),
        "Somewhat Positive": (percentile_values[4], percentile_values[5]),
        "Positive": (percentile_values[5], percentile_values[6]),
        "Extremely Positive": (percentile_values[6], percentile_values[7]),
    },
}


def classify_emotion_7_classes(compound_score, thresholds):
    """Classifica un compound score in una delle 7 classi emotive."""
    for emotion, (lower, upper) in thresholds.items():
        if lower <= compound_score <= upper:
            return emotion
    return "Neutral"  # Fallback


# Test delle configurazioni
results = {}

for config_name, thresholds in configurations.items():
    # Classifica ogni video secondo questa configurazione
    classified_data = []
    for item in video_sentiment_data:
        emotion = classify_emotion_7_classes(item["compound_score"], thresholds)
        classified_data.append(
            {
                "video_name": item["video_name"],
                "caption": item["caption"],
                "compound_score": item["compound_score"],
                "emotion": emotion,
            }
        )

    # Conta la distribuzione
    emotion_counts = Counter([item["emotion"] for item in classified_data])
    results[config_name] = {"data": classified_data, "counts": emotion_counts}

    print(f"\n{config_name}:")
    print("=" * 60)
    for emotion in [
        "Extremely Negative",
        "Negative",
        "Somewhat Negative",
        "Neutral",
        "Somewhat Positive",
        "Positive",
        "Extremely Positive",
    ]:
        count = emotion_counts.get(emotion, 0)
        percentage = (count / len(classified_data)) * 100
        print(f"{emotion:25s}: {count:5d} ({percentage:5.2f}%)")


Config 1 - Uniforme:
Extremely Negative       :    11 ( 0.63%)
Negative                 :    81 ( 4.66%)
Somewhat Negative        :   105 ( 6.04%)
Neutral                  :   772 (44.39%)
Somewhat Positive        :   222 (12.77%)
Positive                 :   392 (22.54%)
Extremely Positive       :   156 ( 8.97%)

Config 2 - Concentrata:
Extremely Negative       :     4 ( 0.23%)
Negative                 :    74 ( 4.26%)
Somewhat Negative        :    98 ( 5.64%)
Neutral                  :   840 (48.30%)
Somewhat Positive        :   275 (15.81%)
Positive                 :   363 (20.87%)
Extremely Positive       :    85 ( 4.89%)

Config 3 - Percentili:
Extremely Negative       :   961 (55.26%)
Negative                 :     0 ( 0.00%)
Somewhat Negative        :     0 ( 0.00%)
Neutral                  :    54 ( 3.11%)
Somewhat Positive        :   253 (14.55%)
Positive                 :   222 (12.77%)
Extremely Positive       :   249 (14.32%)


## Fase 5: Visualizzazione Comparativa delle Configurazioni


In [44]:
# Creazione di un grafico comparativo per le tre configurazioni
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

emotions_order = [
    "Extremely Negative",
    "Negative",
    "Somewhat Negative",
    "Neutral",
    "Somewhat Positive",
    "Positive",
    "Extremely Positive",
]
colors = ["#8B0000", "#DC143C", "#FFA07A", "#808080", "#90EE90", "#32CD32", "#006400"]

for idx, (config_name, result) in enumerate(results.items()):
    ax = axes[idx]
    counts = [result["counts"].get(emotion, 0) for emotion in emotions_order]

    bars = ax.bar(range(len(emotions_order)), counts, color=colors)
    ax.set_title(config_name, fontsize=12, fontweight="bold")
    ax.set_xlabel("Emozione", fontsize=10)
    ax.set_ylabel("Numero di Video", fontsize=10)
    ax.set_xticks(range(len(emotions_order)))
    ax.set_xticklabels(
        [
            "Estr.\nNeg.",
            "Neg.",
            "Po'\nNeg.",
            "Neutro",
            "Po'\nPos.",
            "Pos.",
            "Estr.\nPos.",
        ],
        fontsize=9,
    )
    ax.grid(axis="y", alpha=0.3)

    # Aggiungi i numeri sopra le barre
    for bar in bars:
        height = bar.get_height()
        if height > 0:
            ax.text(
                bar.get_x() + bar.get_width() / 2.0,
                height,
                f"{int(height)}",
                ha="center",
                va="bottom",
                fontsize=9,
            )

plt.tight_layout()
plt.savefig(
    os.path.join(output_folder, f"{mode}_7_classes_comparison.png"),
    dpi=300,
    bbox_inches="tight",
)
plt.close()

print("\nGrafico comparativo salvato!")


Grafico comparativo salvato!


## Fase 6: Selezione della Configurazione Ottimale

Analizzando i risultati, sceglieremo la configurazione che offre la migliore distribuzione per il nostro caso d'uso.


In [45]:
# Scelta della configurazione (modifica questo valore dopo aver analizzato i risultati)
# Opzioni: 'Config 1 - Uniforme', 'Config 2 - Concentrata', 'Config 3 - Percentili'
selected_config = "Config 2 - Concentrata"  # MODIFICA QUESTO VALORE

print(f"\nConfigurazione selezionata: {selected_config}")
print("=" * 60)

# Recupera i dati della configurazione selezionata
final_data = results[selected_config]["data"]
final_counts = results[selected_config]["counts"]

print("\nDistribuzione finale:")
for emotion in emotions_order:
    count = final_counts.get(emotion, 0)
    percentage = (count / len(final_data)) * 100
    print(f"{emotion:25s}: {count:5d} ({percentage:5.2f}%)")

print(f"\nTotale video classificati: {len(final_data)}")


Configurazione selezionata: Config 2 - Concentrata

Distribuzione finale:
Extremely Negative       :     4 ( 0.23%)
Negative                 :    74 ( 4.26%)
Somewhat Negative        :    98 ( 5.64%)
Neutral                  :   840 (48.30%)
Somewhat Positive        :   275 (15.81%)
Positive                 :   363 (20.87%)
Extremely Positive       :    85 ( 4.89%)

Totale video classificati: 1739


## Fase 7: Visualizzazione degli Esempi per Classe


In [46]:
# Mostra esempi rappresentativi per ogni classe
print("\n" + "=" * 80)
print("ESEMPI RAPPRESENTATIVI PER OGNI CLASSE")
print("=" * 80)

for emotion in emotions_order:
    examples = [item for item in final_data if item["emotion"] == emotion]

    if examples:
        print(f"\n{emotion.upper()}:")
        print("-" * 80)

        # Mostra fino a 3 esempi
        for i, example in enumerate(examples[:3], 1):
            print(f"\nEsempio {i}:")
            print(f"  Video: {example['video_name']}")
            print(f"  Compound Score: {example['compound_score']:.4f}")
            print(
                f"  Caption: {example['caption'][:150]}{'...' if len(example['caption']) > 150 else ''}"
            )
    else:
        print(f"\n{emotion.upper()}: Nessun esempio trovato")


ESEMPI RAPPRESENTATIVI PER OGNI CLASSE

EXTREMELY NEGATIVE:
--------------------------------------------------------------------------------

Esempio 1:
  Video: DfnHNkTE7mE_15-5-rgb_front
  Compound Score: -0.7964
  Caption: So I was an ex-KGB guy running around trying to kill Crockett and Tubbs with a machine gun for most of the show.

Esempio 2:
  Video: bbSKqkE4EBs_6-8-rgb_front
  Compound Score: -0.7579
  Caption: That action of the diaphragm actually moves two thirds of the lymphatic fluids of our body, lymph being, I hate to say it, the gook of our body, or th...

Esempio 3:
  Video: cwLJfXFf9ks_18-8-rgb_front
  Compound Score: -0.8555
  Caption: If it is negative energy or anger or frustration you don't need to deal with it.

NEGATIVE:
--------------------------------------------------------------------------------

Esempio 1:
  Video: -f1_kdl050s_0-1-rgb_front
  Compound Score: -0.4939
  Caption: In this clip we are going to talk about dangers for these birds in the household

## Fase 8: Salvataggio dei Risultati


In [47]:
# Salvataggio dei dati classificati in CSV
output_csv_path = f"../../data/processed/{mode}/video_sentiment_data_7_classes.csv"

# Crea la directory se non esiste
os.makedirs(os.path.dirname(output_csv_path), exist_ok=True)

# Prepara i dati per il salvataggio
csv_data = [
    {
        "video_name": item["video_name"],
        "caption": item["caption"],
        "compound_score": item["compound_score"],
        "emotion": item["emotion"],
    }
    for item in final_data
]

# Scrivi il CSV
with open(output_csv_path, mode="w", newline="", encoding="utf-8") as csv_file:
    fieldnames = ["video_name", "caption", "compound_score", "emotion"]
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(csv_data)

print(f"\nFile CSV salvato con successo in: {output_csv_path}")
print(f"Numero di righe salvate: {len(csv_data)}")


File CSV salvato con successo in: ../../data/processed/val/video_sentiment_data_7_classes.csv
Numero di righe salvate: 1739


In [48]:
# Salvataggio delle soglie utilizzate per riferimento futuro
thresholds_path = f"../../data/processed/{mode}/thresholds_7_classes.txt"

with open(thresholds_path, "w", encoding="utf-8") as f:
    f.write(f"Configurazione utilizzata: {selected_config}\n")
    f.write(f"Data: {pd.Timestamp.now()}\n")
    f.write(f"Modalità: {mode}\n")
    f.write("\nSoglie:\n")
    f.write("=" * 60 + "\n")

    selected_thresholds = configurations[selected_config]
    for emotion, (lower, upper) in selected_thresholds.items():
        f.write(f"{emotion:25s}: {lower:7.4f} < compound ≤ {upper:7.4f}\n")

    f.write("\nDistribuzione:\n")
    f.write("=" * 60 + "\n")
    for emotion in emotions_order:
        count = final_counts.get(emotion, 0)
        percentage = (count / len(final_data)) * 100
        f.write(f"{emotion:25s}: {count:5d} ({percentage:5.2f}%)\n")

print(f"\nSoglie salvate in: {thresholds_path}")


Soglie salvate in: ../../data/processed/val/thresholds_7_classes.txt


## 📊 Spiegazione Dettagliata delle Tre Configurazioni di Soglie

### **Configurazione 1 - Divisione Uniforme**

**Filosofia:** Divisione matematicamente equa dello spazio dei compound scores [-1, +1]

**Soglie:**

```
Estremamente Negativo: -1.00 ≤ compound ≤ -0.67
Negativo:             -0.67 < compound ≤ -0.33
Un po' Negativo:      -0.33 < compound ≤ -0.05
Neutro:               -0.05 < compound ≤  0.05
Un po' Positivo:       0.05 < compound ≤  0.33
Positivo:              0.33 < compound ≤  0.67
Estremamente Positivo: 0.67 < compound ≤  1.00
```

**✅ Vantaggi:**

- Semplice da comprendere e implementare
- Matematicamente bilanciata (ogni intervallo ha ampiezza simile)
- Intuitiva: ogni classe occupa circa 1/7 dello spettro
- Facile da replicare su diversi dataset

**❌ Svantaggi:**

- Non tiene conto della distribuzione reale dei dati
- Potrebbe classificare come "estremi" casi che non lo sono semanticamente
- Ignora il fatto che i compound scores potrebbero concentrarsi in certe zone
- L'area neutrale potrebbe essere troppo ristretta (solo 0.10)

**💡 Quando usarla:** Quando non hai informazioni sulla distribuzione dei dati o vuoi un approccio teorico e replicabile

---

### **Configurazione 2 - Concentrata sul Neutro** ⭐ (SCELTA CONSIGLIATA)

**Filosofia:** Riconoscere che molte frasi sono veramente neutre, riservando le etichette "estreme" solo ai casi più intensi

**Soglie:**

```
Estremamente Negativo: -1.00 ≤ compound ≤ -0.75
Negativo:             -0.75 < compound ≤ -0.40
Un po' Negativo:      -0.40 < compound ≤ -0.10
Neutro:               -0.10 < compound ≤  0.10  ← AREA PIÙ AMPIA
Un po' Positivo:       0.10 < compound ≤  0.40
Positivo:              0.40 < compound ≤  0.75
Estremamente Positivo: 0.75 < compound ≤  1.00
```

**✅ Vantaggi:**

- **Area neutrale più ampia** (0.20 invece di 0.10): riflette la realtà linguistica
- **Soglie estreme più conservative** (±0.75): solo le frasi davvero intense vengono classificate come "estremamente" positive/negative
- **Migliore interpretabilità**: le etichette corrispondono meglio all'intensità percepita
- **Riduce falsi positivi** nelle classi estreme
- **Transizioni più naturali** tra le classi intermedie
- **Maggiore significato semantico**: ogni etichetta riflette meglio l'intensità reale

**❌ Svantaggi:**

- Potrebbe risultare in classi sbilanciate (molti neutri, pochi estremi)
- Richiede più dati nelle classi estreme per l'addestramento
- Potrebbe necessitare di data augmentation per le classi meno rappresentate

**💡 Quando usarla:** Quando vuoi precisione semantica e hai un dataset dove molte frasi sono effettivamente neutre o ambigue (CONSIGLIATA per analisi linguistica e NLP)

---

### **Configurazione 3 - Basata sui Percentili**

**Filosofia:** Garantire che ogni classe contenga esattamente il 14.28% dei dati (1/7)

**Soglie:** Calcolate dinamicamente sui percentili: 0%, 14.28%, 28.57%, 42.86%, 57.14%, 71.43%, 85.71%, 100%

**Esempio (dipende dai tuoi dati):**

```
Estremamente Negativo: -1.00 ≤ compound ≤ -0.52  (14.28% dei dati)
Negativo:             -0.52 < compound ≤ -0.23  (14.28% dei dati)
Un po' Negativo:      -0.23 < compound ≤ -0.05  (14.28% dei dati)
Neutro:               -0.05 < compound ≤  0.15  (14.28% dei dati)
Un po' Positivo:       0.15 < compound ≤  0.38  (14.28% dei dati)
Positivo:              0.38 < compound ≤  0.62  (14.28% dei dati)
Estremamente Positivo: 0.62 < compound ≤  1.00  (14.28% dei dati)
```

**✅ Vantaggi:**

- **Classi perfettamente bilanciate**: ogni classe ha lo stesso numero di esempi
- **Ottimale per machine learning**: evita problemi di class imbalance
- **Adattiva ai dati**: le soglie si adattano alla distribuzione effettiva
- **Massima rappresentazione** di tutte le classi nel training set

**❌ Svantaggi:**

- **Perde significato semantico**: una frase "moderatamente positiva" potrebbe finire in "estremamente positivo" solo perché è nel top 14%
- **Non generalizzabile**: le soglie cambiano tra train/val/test se le distribuzioni differiscono
- **Dipendente dal dataset**: difficile confrontare risultati con altri studi
- **Può ingannare**: le etichette non riflettono l'intensità reale del sentiment
- **Problemi di overfitting**: il modello impara soglie specifiche del training set

**💡 Quando usarla:** Quando l'obiettivo principale è il bilanciamento perfetto delle classi per training di modelli, e il significato semantico è secondario

---

## 🎯 Confronto Riassuntivo

| Aspetto                     | Config 1 (Uniforme) | Config 2 (Concentrata) ⭐ | Config 3 (Percentili) |
| --------------------------- | ------------------- | ------------------------- | --------------------- |
| **Interpretabilità**        | Media               | Alta                      | Bassa                 |
| **Bilanciamento classi**    | Variabile           | Sbilanciato               | Perfetto              |
| **Significato semantico**   | Medio               | Alto                      | Basso                 |
| **Generalizzabilità**       | Alta                | Alta                      | Bassa                 |
| **Per ML training**         | Buona               | Media                     | Ottima                |
| **Per analisi linguistica** | Media               | Ottima                    | Sconsigliata          |
| **Replicabilità**           | Alta                | Alta                      | Bassa                 |
| **Area Neutrale**           | Stretta (0.10)      | Ampia (0.20)              | Variabile             |

---

## 🔍 Perché Config 2 è Stata Scelta?

Nel contesto dell'analisi del sentiment per il progetto EmoSign:

1. **Precisione semantica** > Bilanciamento perfetto

   - È più importante che le etichette abbiano senso piuttosto che avere lo stesso numero di esempi per classe

2. **Natura del linguaggio dei segni**

   - Le frasi nella lingua dei segni spesso hanno un tono veramente neutro (descrizioni, narrazioni)
   - Non tutte le conversazioni sono cariche emotivamente

3. **Significato delle classi "estreme"**

   - È importante che le classi "estreme" rappresentino davvero emozioni intense
   - Soglie a ±0.75 garantiscono che solo frasi molto cariche finiscano in queste categorie

4. **Generalizzabilità**

   - La configurazione è replicabile su altri dataset senza perdere significato
   - Le soglie sono indipendenti dalla distribuzione specifica del training set

5. **Qualità > Quantità**

   - Meglio avere pochi esempi ben etichettati nelle classi estreme che molti esempi ambigui
   - Facilita l'interpretazione dei risultati del modello

6. **Validazione e Test**
   - Le stesse soglie possono essere applicate ai set di validazione e test mantenendo coerenza
   - Non c'è rischio di "data leakage" come con i percentili

---

## 📈 Implicazioni Pratiche

### Per il Training del Modello:

- Potrebbero essere necessarie tecniche di **data augmentation** per le classi meno rappresentate
- Usare **class weights** durante il training per compensare lo sbilanciamento
- Valutare con **F1-score macro** per dare uguale importanza a tutte le classi

### Per l'Interpretazione:

- Le predizioni del modello avranno un **significato chiaro** e interpretabile
- Più facile spiegare i risultati agli stakeholder
- Le analisi qualitative saranno più affidabili
